In [1]:
# Set your paths here (run this cell before running the next cell)
import os
input_video_path = r"C:\Users\Roemmich\Downloads\input_video.MOV" #this is the name and location of the video that you want to track
output_video_path = r"C:\Users\Roemmich\Downloads\input_video_pose.avi" #this is the name and location of your desired output video that will have your tracked landmarks overlaid on top of the input video
output_csv_path = r"C:\Users\Roemmich\Downloads\input_video_outputs.csv" #this is the name and location of your desired output .csv file where you will save the frame-by-frame pixel coordinate data
output_images_path = r"C:\Users\Roemmich\Downloads\input_video_images" #this is the name of the folder where you want to save the frame-by-frame tracked images in case you need to visually inspect the tracking results frame-by-frame
if os.path.isdir(output_images_path) is False:
    os.mkdir(output_images_path)

In [2]:
# Run to this cell perform Google Mediapipe hand tracking. Be sure that you have previously installed Mediapipe and OpenCV.
import mediapipe as mp
import cv2
import numpy as np
import time

print('If you want to stop the analysis at any time, press "q" for quit')

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

#define landmarks and save as .csv
landmark_names = np.array(['L_WRIST_X','L_WRIST_Y','L_THUMB_1_X','L_THUMB_1_Y','L_THUMB_2_X','L_THUMB_2_Y','L_THUMB_3_X','L_THUMB_3_Y','L_THUMB_4_X','L_THUMB_4_Y','L_INDEX_1_X','L_INDEX_1_Y','L_INDEX_2_X','L_INDEX_2_Y','L_INDEX_3_X','L_INDEX_3_Y','L_INDEX_4_X','L_INDEX_4_Y','L_MIDDLE_1_X','L_MIDDLE_1_Y','L_MIDDLE_2_X','L_MIDDLE_2_Y','L_MIDDLE_3_X','L_MIDDLE_3_Y','L_MIDDLE_4_X','L_MIDDLE_4_Y','L_RING_1_X','L_RING_1_Y','L_RING_2_X','L_RING_2_Y','L_RING_3_X','L_RING_3_Y','L_RING_4_X','L_RING_4_Y','L_PINKY_1_X','L_PINKY_1_Y','L_PINKY_2_X','L_PINKY_2_Y','L_PINKY_3_X','L_PINKY_3_Y','L_PINKY_4_X','L_PINKY_4_Y','R_WRIST_X','R_WRIST_Y','R_THUMB_1_X','R_THUMB_1_Y','R_THUMB_2_X','R_THUMB_2_Y','R_THUMB_3_X','R_THUMB_3_Y','R_THUMB_4_X','R_THUMB_4_Y','R_INDEX_1_X','R_INDEX_1_Y','R_INDEX_2_X','R_INDEX_2_Y','R_INDEX_3_X','R_INDEX_3_Y','R_INDEX_4_X','R_INDEX_4_Y','R_MIDDLE_1_X','R_MIDDLE_1_Y','R_MIDDLE_2_X','R_MIDDLE_2_Y','R_MIDDLE_3_X','R_MIDDLE_3_Y','R_MIDDLE_4_X','R_MIDDLE_4_Y','R_RING_1_X','R_RING_1_Y','R_RING_2_X','R_RING_2_Y','R_RING_3_X','R_RING_3_Y','R_RING_4_X','R_RING_4_Y','R_PINKY_1_X','R_PINKY_1_Y','R_PINKY_2_X','R_PINKY_2_Y','R_PINKY_3_X','R_PINKY_3_Y','R_PINKY_4_X','R_PINKY_4_Y'])
len_landmarks = len(landmark_names)
landmark_names.shape = (1,len_landmarks)
np.savetxt(output_csv_path,landmark_names,fmt='%s',delimiter=',')

#initializing data array
data = np.array([])
data = np.array([])

#set iteration counter
iter = 0

#bring in video, define video output
cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width, frame_height))

# Run hand tracking
with mp_hands.Hands(min_detection_confidence=0.2,min_tracking_confidence=0.2) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print_iter = str(iter)
            print("Tracking complete, " + print_iter + ' frames analyzed')
            print("Your tracked output video has been saved to " + output_video_path)
            print("Your output .csv data file has been saved to " + output_csv_path)
            print("Your frame-by-frame tracked images have been saved to " + output_images_path)
            break
    
        # Convert image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image
        image_height, image_width, _ = image.shape
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_name = str(output_images_path + "\Image" + str(iter + 1) + '.png')
        
        if results.multi_hand_landmarks is None:
            data=np.concatenate((data,np.zeros((len_landmarks,),dtype=int)),axis=None)
        elif results.multi_hand_landmarks:
            if len(results.multi_hand_landmarks) == 1:
                for idx, hand_handedness in enumerate(results.multi_handedness):
                    which_hand = hand_handedness.classification[0].index
                if which_hand == 1:
                    for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(0,0,0),thickness=2,circle_radius=1),mp_drawing.DrawingSpec(color=(0,0,255),thickness=2,circle_radius=1))
                        for point in mp_hands.HandLandmark:
                            normalized_landmark = hand_landmarks.landmark[point]
                            pixel_coordinates_landmark = [0,0] if mp_drawing._normalized_to_pixel_coordinates(normalized_landmark.x, normalized_landmark.y, image_width, image_height) is None else mp_drawing._normalized_to_pixel_coordinates(normalized_landmark.x, normalized_landmark.y, image_width, image_height)
                            data=np.concatenate((data,pixel_coordinates_landmark[0], pixel_coordinates_landmark[1]),axis=None)
                        data=np.concatenate((data,np.zeros((len_landmarks // 2,),dtype=int)),axis=None)
                elif which_hand == 0:
                    for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(0,0,0),thickness=2,circle_radius=1),mp_drawing.DrawingSpec(color=(0,0,255),thickness=2,circle_radius=1))
                        data=np.concatenate((data,np.zeros((len_landmarks // 2,),dtype=int)),axis=None)
                        for point in mp_hands.HandLandmark:
                            normalized_landmark = hand_landmarks.landmark[point]
                            pixel_coordinates_landmark = [0,0] if mp_drawing._normalized_to_pixel_coordinates(normalized_landmark.x, normalized_landmark.y, image_width, image_height) is None else mp_drawing._normalized_to_pixel_coordinates(normalized_landmark.x, normalized_landmark.y, image_width, image_height)
                            data=np.concatenate((data,pixel_coordinates_landmark[0], pixel_coordinates_landmark[1]),axis=None)
            else:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(0,0,0),thickness=2,circle_radius=1),mp_drawing.DrawingSpec(color=(0,0,255),thickness=2,circle_radius=1))
                    for point in mp_hands.HandLandmark:
                        normalized_landmark = hand_landmarks.landmark[point]
                        pixel_coordinates_landmark = [0,0] if mp_drawing._normalized_to_pixel_coordinates(normalized_landmark.x, normalized_landmark.y, image_width, image_height) is None else mp_drawing._normalized_to_pixel_coordinates(normalized_landmark.x, normalized_landmark.y, image_width, image_height)
                        data=np.concatenate((data,pixel_coordinates_landmark[0], pixel_coordinates_landmark[1]),axis=None)
                
        out.write(image)
        cv2.imshow('MediaPipe Hands', image)
        save_image = cv2.imwrite(image_name, image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            print("Analysis stopped")
            break
        iter = iter+1

# Close video display
cap.release()
out.release()
cv2.destroyAllWindows()

#organize data for saving
data.shape = (iter,len_landmarks)

#save data into .csv
with open(output_csv_path,'a',newline='') as csvfile:
    np.savetxt(csvfile,data,fmt='%f',delimiter=',')

If you want to stop the analysis at any time, press "q" for quit
Tracking complete, 612 frames analyzed
Your tracked output video has been saved to C:\Users\Roemmich\Downloads\stroke_for_Joan\Day_30_paretic_pose.avi
Your output .csv data file has been saved to C:\Users\Roemmich\Downloads\stroke_for_Joan\Day_30_paretic_outputs.csv
Your frame-by-frame tracked images have been saved to C:\Users\Roemmich\Downloads\stroke_for_Joan\Day_30_paretic_images
